In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

In [5]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [6]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

In [7]:
ngrams

[(['forty', 'When'], 'winters'),
 (['winters', 'forty'], 'shall'),
 (['shall', 'winters'], 'besiege'),
 (['besiege', 'shall'], 'thy'),
 (['thy', 'besiege'], 'brow,'),
 (['brow,', 'thy'], 'And'),
 (['And', 'brow,'], 'dig'),
 (['dig', 'And'], 'deep'),
 (['deep', 'dig'], 'trenches'),
 (['trenches', 'deep'], 'in'),
 (['in', 'trenches'], 'thy'),
 (['thy', 'in'], "beauty's"),
 (["beauty's", 'thy'], 'field,'),
 (['field,', "beauty's"], 'Thy'),
 (['Thy', 'field,'], "youth's"),
 (["youth's", 'Thy'], 'proud'),
 (['proud', "youth's"], 'livery'),
 (['livery', 'proud'], 'so'),
 (['so', 'livery'], 'gazed'),
 (['gazed', 'so'], 'on'),
 (['on', 'gazed'], 'now,'),
 (['now,', 'on'], 'Will'),
 (['Will', 'now,'], 'be'),
 (['be', 'Will'], 'a'),
 (['a', 'be'], "totter'd"),
 (["totter'd", 'a'], 'weed'),
 (['weed', "totter'd"], 'of'),
 (['of', 'weed'], 'small'),
 (['small', 'of'], 'worth'),
 (['worth', 'small'], 'held:'),
 (['held:', 'worth'], 'Then'),
 (['Then', 'held:'], 'being'),
 (['being', 'Then'], 'asked

In [8]:

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [9]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [10]:
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

In [11]:
model(torch.tensor([1,2]))

tensor([[-4.3716, -4.6812, -4.7513, -4.2395, -4.5980, -4.4615, -4.3640, -4.5541,
         -4.4755, -4.3974, -4.5871, -4.2178, -4.4652, -4.6792, -4.6893, -4.5764,
         -4.3727, -4.3335, -4.7382, -4.5967, -4.4715, -4.6345, -4.6357, -4.9677,
         -4.9105, -5.2268, -4.3867, -4.5053, -5.1275, -4.3163, -4.7395, -4.7795,
         -4.6904, -4.7321, -4.7658, -4.8316, -4.4296, -4.8624, -4.6846, -4.7190,
         -4.7234, -4.3837, -4.8154, -4.8126, -4.6532, -4.2744, -4.7025, -4.6811,
         -4.2640, -4.7771, -4.4732, -4.6610, -4.4694, -4.3045, -4.8156, -4.9493,
         -4.4761, -4.8769, -4.8578, -4.6262, -4.7998, -4.6201, -4.8968, -4.3798,
         -4.6686, -4.6119, -4.7188, -4.4074, -4.0597, -4.1301, -4.1384, -4.3515,
         -4.5970, -4.1615, -4.7674, -4.3190, -4.8796, -4.4806, -4.7584, -4.3875,
         -4.2433, -4.8221, -4.7873, -5.1393, -4.6367, -4.9584, -4.7859, -4.7477,
         -4.4687, -4.5424, -4.3140, -4.3382, -4.7881, -4.8354, -4.5693, -4.8900,
         -4.3149]], grad_fn=

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [13]:
log_probs = model(torch.tensor([1,2]))

In [14]:
loss = loss_function(log_probs, torch.tensor([3], dtype=torch.long))

In [15]:
loss2 = loss_function(log_probs, torch.tensor([68], dtype=torch.long))

In [16]:
loss

tensor(4.2395, grad_fn=<NllLossBackward>)

In [17]:
loss2

tensor(4.0597, grad_fn=<NllLossBackward>)

In [18]:
torch.argmax(log_probs)

tensor(68)

In [19]:
log_probs[0,3]

tensor(-4.2395, grad_fn=<SelectBackward>)

In [20]:
log_probs[0,68]

tensor(-4.0597, grad_fn=<SelectBackward>)

In [21]:
loss.backward()
optimizer.step()